In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

python_gradient = model._gradient(params)

print("Gradient (C++):", gradient_ctypes)
print("Gradient (Python):", python_gradient)

are_close = np.allclose(gradient_ctypes, python_gradient)
differences = gradient_ctypes - python_gradient

print('Are the gradients approximately equal?', are_close)
print('Differences:', differences)

Gradient (C++): [-3.84996821e+06 -7.58983307e+06 -3.61117750e+06 -3.37967640e+06
 -3.15421533e+06 -2.93566312e+06 -2.72478215e+06 -2.52117447e+06
 -2.32481428e+06 -2.13600990e+06 -1.95538892e+06 -1.78216026e+06
 -1.61593357e+06 -1.45821237e+06 -1.30879223e+06 -1.16686926e+06
 -1.03280968e+06 -9.07147042e+05 -7.90414623e+05 -6.81641055e+05
 -5.81069524e+05 -4.89204407e+05 -4.05458080e+05 -3.29063207e+05
 -2.60692991e+05 -2.01032683e+05 -1.49393999e+05 -4.61106970e+05
 -3.17982868e+04 -2.49577914e+04 -5.16569854e+04 -2.76570782e+04
 -2.44294880e+05 -3.01011508e+03 -5.37259506e+04  7.54740261e+04
  2.36191622e+02  1.72682692e+04 -1.92233114e+03  9.40767260e+02
  3.39320773e+04 -1.62798437e+04 -5.72984893e+04 -2.53433308e+04
  6.42428550e+03  3.90416906e+03  5.22863060e+04]
Gradient (Python): [-3.84996821e+06 -7.58983307e+06 -3.61119750e+06 -3.37969640e+06
 -3.15423533e+06 -2.93568312e+06 -2.72480215e+06 -2.52119447e+06
 -2.32483428e+06 -2.13602990e+06 -1.95540892e+06 -1.78218026e+06
 -1.6

In [2]:
print('Are the gradients approximately equal?', are_close)

Are the gradients approximately equal? False


In [3]:
differences

array([ 0.00000000e+00, -3.72529030e-09,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  5.23868948e-10,
       -6.84049537e+03,  2.69928671e+03, -4.86468703e+04, -1.03131104e+05,
       -2.61563149e+05, -3.95088234e+03, -3.74461069e+04,  1.00817357e+05,
       -3.66797744e+03,  4.90665560e+04,  4.97346542e+04,  2.45235647e+05,
        8.76580279e+04, -1.65160353e+04, -5.53761582e+04, -5.92754081e+04,
        6.37227748e+04, -2.52011645e+03, -5.09317033e-11])

In [4]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_2.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

python_gradient = model._gradient(params)

print("Gradient (C++):", gradient_ctypes)
print("Gradient (Python):", python_gradient)

are_close = np.allclose(gradient_ctypes, python_gradient)
differences = gradient_ctypes - python_gradient

print('Are the gradients approximately equal?', are_close)
print('Differences:', differences)

Gradient (C++): [-4.21793487e+04 -8.31524309e+04 -3.95434328e+04 -3.70071610e+04
 -3.45370625e+04 -3.21426557e+04 -2.98322932e+04 -2.76016152e+04
 -2.54503391e+04 -2.33818426e+04 -2.14030015e+04 -1.95051489e+04
 -1.76840082e+04 -1.59560518e+04 -1.43190398e+04 -1.27641651e+04
 -1.12954397e+04 -9.91870933e+03 -8.63981632e+03 -7.44811863e+03
 -6.34628053e+03 -5.33982785e+03 -4.42232274e+03 -3.58535842e+03
 -2.83631038e+03 -2.18268603e+03 -1.61694473e+03 -5.05177981e+03
 -3.48374572e+02 -2.73431709e+02 -5.65941815e+02 -3.03004461e+02
 -2.67643740e+03 -3.29781148e+01 -5.88608913e+02  8.26875727e+02
  2.58766001e+00  1.89187107e+02 -2.10606091e+01  1.03068254e+01
  3.71751880e+02 -1.78358149e+02 -6.27748809e+02 -2.77655588e+02
  7.03829651e+01  4.27731605e+01  5.72836504e+02]
Gradient (Python): [-4.21793487e+04 -8.31524309e+04 -3.95634328e+04 -3.70271610e+04
 -3.45570625e+04 -3.21626557e+04 -2.98522932e+04 -2.76216152e+04
 -2.54703391e+04 -2.34018426e+04 -2.14230015e+04 -1.95251489e+04
 -1.7

In [5]:
differences

array([ 0.00000000e+00, -4.36557457e-11,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  6.36646291e-12,
       -7.49428627e+01,  2.95727521e+01, -5.32963700e+02, -1.12988019e+03,
       -2.86562451e+03, -4.32849403e+01, -4.10250764e+02,  1.10453131e+03,
       -4.01855004e+01,  5.37561679e+02,  5.44881206e+02,  2.68674423e+03,
        9.60360793e+02, -1.80945809e+02, -6.06688200e+02, -6.49407468e+02,
        6.98131775e+02, -2.76098047e+01, -5.68434189e-13])

In [6]:
gradient_ctypes

array([-4.21793487e+04, -8.31524309e+04, -3.95434328e+04, -3.70071610e+04,
       -3.45370625e+04, -3.21426557e+04, -2.98322932e+04, -2.76016152e+04,
       -2.54503391e+04, -2.33818426e+04, -2.14030015e+04, -1.95051489e+04,
       -1.76840082e+04, -1.59560518e+04, -1.43190398e+04, -1.27641651e+04,
       -1.12954397e+04, -9.91870933e+03, -8.63981632e+03, -7.44811863e+03,
       -6.34628053e+03, -5.33982785e+03, -4.42232274e+03, -3.58535842e+03,
       -2.83631038e+03, -2.18268603e+03, -1.61694473e+03, -5.05177981e+03,
       -3.48374572e+02, -2.73431709e+02, -5.65941815e+02, -3.03004461e+02,
       -2.67643740e+03, -3.29781148e+01, -5.88608913e+02,  8.26875727e+02,
        2.58766001e+00,  1.89187107e+02, -2.10606091e+01,  1.03068254e+01,
        3.71751880e+02, -1.78358149e+02, -6.27748809e+02, -2.77655588e+02,
        7.03829651e+01,  4.27731605e+01,  5.72836504e+02])

In [7]:
python_gradient

array([-4.21793487e+04, -8.31524309e+04, -3.95634328e+04, -3.70271610e+04,
       -3.45570625e+04, -3.21626557e+04, -2.98522932e+04, -2.76216152e+04,
       -2.54703391e+04, -2.34018426e+04, -2.14230015e+04, -1.95251489e+04,
       -1.77040082e+04, -1.59760518e+04, -1.43390398e+04, -1.27841651e+04,
       -1.13154397e+04, -9.93870933e+03, -8.65981632e+03, -7.46811863e+03,
       -6.36628053e+03, -5.35982785e+03, -4.44232274e+03, -3.60535842e+03,
       -2.85631038e+03, -2.20268603e+03, -1.63694473e+03, -5.05177981e+03,
       -2.73431709e+02, -3.03004461e+02, -3.29781148e+01,  8.26875727e+02,
        1.89187107e+02,  1.03068254e+01, -1.78358149e+02, -2.77655588e+02,
        4.27731605e+01, -3.48374572e+02, -5.65941815e+02, -2.67643740e+03,
       -5.88608913e+02,  2.58766001e+00, -2.10606091e+01,  3.71751880e+02,
       -6.27748809e+02,  7.03829651e+01,  5.72836504e+02])

In [9]:
differences

array([ 0.00000000e+00, -4.36557457e-11,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  2.00000000e+01,
        2.00000000e+01,  2.00000000e+01,  2.00000000e+01,  6.36646291e-12,
       -7.49428627e+01,  2.95727521e+01, -5.32963700e+02, -1.12988019e+03,
       -2.86562451e+03, -4.32849403e+01, -4.10250764e+02,  1.10453131e+03,
       -4.01855004e+01,  5.37561679e+02,  5.44881206e+02,  2.68674423e+03,
        9.60360793e+02, -1.80945809e+02, -6.06688200e+02, -6.49407468e+02,
        6.98131775e+02, -2.76098047e+01, -5.68434189e-13])

In [11]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_3.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

print("Gradient (C++):", gradient_ctypes)

Gradient (C++): [-5.33812791e+05 -1.05235933e+06 -5.00686320e+05 -4.68587808e+05
 -4.37326771e+05 -4.07023673e+05 -3.77784220e+05 -3.49553238e+05
 -3.22327149e+05 -2.96148701e+05 -2.71104913e+05 -2.47086097e+05
 -2.24038131e+05 -2.02169486e+05 -1.81451813e+05 -1.61773651e+05
 -1.43185779e+05 -1.25762174e+05 -1.09576778e+05 -9.44949076e+04
 -6.39342418e+04 -4.40895386e+03 -3.46049307e+03 -7.16243823e+03
 -3.83475947e+03 -3.38724177e+04 -4.17363948e+02 -7.44930816e+03
  1.04647619e+04  3.27488701e+01  2.39431145e+03 -2.66538552e+02
  1.30440972e+02  4.70481206e+03 -2.25726248e+03 -7.94465430e+03
 -3.51394957e+03  8.90751711e+02  5.41327945e+02  7.24969593e+03
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00]


In [12]:
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)
gradient_ctypes
#gradient_ctypes[-20:]

array([-5.33812791e+05, -1.05235933e+06, -5.00686320e+05, -4.68587808e+05,
       -4.37326771e+05, -4.07023673e+05, -3.77784220e+05, -3.49553238e+05,
       -3.22327149e+05, -2.96148701e+05, -2.71104913e+05, -2.47086097e+05,
       -2.24038131e+05, -2.02169486e+05, -1.81451813e+05, -1.61773651e+05,
       -1.43185779e+05, -1.25762174e+05, -1.09576778e+05, -9.44949076e+04,
       -6.39342418e+04, -4.40895386e+03, -3.46049307e+03, -7.16243823e+03,
       -3.83475947e+03, -3.38724177e+04, -4.17363948e+02, -7.44930816e+03,
        1.04647619e+04,  3.27488701e+01,  2.39431145e+03, -2.66538552e+02,
        1.30440972e+02,  4.70481206e+03, -2.25726248e+03, -7.94465430e+03,
       -3.51394957e+03,  8.90751711e+02,  5.41327945e+02,  7.24969593e+03,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00])

In [15]:
gradient_ctypes[-20:]

array([-7449.30815557, 10464.76186092,    32.74887009,  2394.31145372,
        -266.53855151,   130.44097216,  4704.81206165, -2257.26248016,
       -7944.65429543, -3513.94956707,   890.75171123,   541.32794501,
        7249.69593436,     0.        ,     0.        ,     0.        ,
           0.        ,     0.        ,     0.        ,     0.        ])

In [14]:
model._gradient(params)[-20:]

array([-6.39342418e+04, -3.46049307e+03, -3.83475947e+03, -4.17363948e+02,
        1.04647619e+04,  2.39431145e+03,  1.30440972e+02, -2.25726248e+03,
       -3.51394957e+03,  5.41327945e+02, -4.40895386e+03, -7.16243823e+03,
       -3.38724177e+04, -7.44930816e+03,  3.27488701e+01, -2.66538552e+02,
        4.70481206e+03, -7.94465430e+03,  8.90751711e+02,  7.24969593e+03])

In [18]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_4.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

1

In [21]:
gradient_ctypes[2:27]

array([-661473.84028616, -619067.80866804, -577768.18312425,
       -537734.1111955 , -499105.24485184, -461808.6892547 ,
       -425839.71987421, -391254.81222521, -358168.92710689,
       -326437.15584371, -295987.99440272, -267096.85847869,
       -239726.2972889 , -213729.05709127, -189172.22024507,
       -166153.52274285, -144770.65011465, -124845.66812736,
       -106423.1232611 ,  -89595.40645151,  -74254.8775834 ,
        -60260.97714784,  -47737.0217845 ,  -36808.53435374,
        -27349.43632336])

In [22]:
model._gradient(params)[2:27]

array([-661493.84028616, -619087.80866804, -577788.18312425,
       -537754.1111955 , -499125.24485184, -461828.6892547 ,
       -425859.71987421, -391274.81222521, -358188.92710689,
       -326457.15584371, -296007.99440272, -267116.85847869,
       -239746.2972889 , -213749.05709127, -189192.22024507,
       -166173.52274285, -144790.65011465, -124865.66812736,
       -106443.1232611 ,  -89615.40645151,  -74274.8775834 ,
        -60280.97714784,  -47757.0217845 ,  -36828.53435374,
        -27369.43632336])

In [24]:
gradient_ctypes[2:27] - model._gradient(params)[2:27]

array([20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.,
       20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20., 20.])

In [8]:
def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

## instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
gradient_ctypes = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    gradient_ctypes
)

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)
cpp_grad = gradient_ctypes

In [10]:
numerical_grad[-20:] # finite differences

array([-6.90807159e+04, -3.73905021e+03, -4.14344372e+03, -4.50960215e+02,
        1.13071372e+04,  2.58704484e+03,  1.40941003e+02, -2.43896394e+03,
       -3.79680973e+03,  5.84902882e+02, -4.76385863e+03, -7.73898850e+03,
       -3.65990242e+04, -8.04895040e+03,  3.53850482e+01, -2.87993957e+02,
        5.08353233e+03, -8.58417005e+03,  9.62454011e+02,  7.83327012e+03])

In [11]:
gradient_ctypes[-20:] # C++ gradient

array([-2.05468197e+04, -1.41692429e+03, -1.11211340e+03, -2.30182329e+03,
       -1.23239299e+03, -1.08857232e+04, -1.34130030e+02, -2.39401590e+03,
        3.36310511e+03,  1.05246439e+01,  7.69470073e+02, -8.56586299e+01,
        4.19203710e+01,  1.51200550e+03, -7.25426061e+02, -2.55320740e+03,
       -1.12929294e+03,  2.86264673e+02,  1.73968868e+02,  2.32986567e+03])

In [12]:
model._gradient(params)[-20:] # Python method gradient

array([-6.90807159e+04, -3.73905018e+03, -4.14344367e+03, -4.50960229e+02,
        1.13071372e+04,  2.58704482e+03,  1.40940996e+02, -2.43896390e+03,
       -3.79680973e+03,  5.84902877e+02, -4.76385862e+03, -7.73898847e+03,
       -3.65990242e+04, -8.04895038e+03,  3.53850351e+01, -2.87993936e+02,
        5.08353233e+03, -8.58417009e+03,  9.62453986e+02,  7.83327011e+03])

In [5]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_4.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
cpp_grad = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_grad
)

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)

In [2]:
numerical_grad # finite differences

array([-7.01413296e+04, -1.38276722e+05, -6.57912430e+04, -6.15735990e+04,
       -5.74659966e+04, -5.34842643e+04, -4.96422918e+04, -4.59328291e+04,
       -4.23554062e+04, -3.89156402e+04, -3.56249648e+04, -3.24689676e+04,
       -2.94405370e+04, -2.65670654e+04, -2.38448281e+04, -2.12591793e+04,
       -1.88167910e+04, -1.65273839e+04, -1.44006735e+04, -1.24189630e+04,
       -1.05866827e+04, -8.91302174e+03, -7.38727441e+03, -5.99546084e+03,
       -4.74984593e+03, -3.66291401e+03, -2.72212548e+03, -8.40075923e+03,
       -4.54697962e+02, -5.03875392e+02, -5.48403150e+01,  1.37503695e+03,
        3.14605033e+02,  1.71395368e+01, -2.96597227e+02, -4.61721975e+02,
        7.11288012e+01, -5.79322736e+02, -9.41121965e+02, -4.45072965e+03,
       -9.78815773e+02,  4.30309956e+00, -3.50223345e+01,  6.18197577e+02,
       -1.04390270e+03,  1.17041986e+02,  9.52587354e+02])

In [3]:
python_grad # Python method gradient

array([-7.01413296e+04, -1.38276722e+05, -6.57912430e+04, -6.15735990e+04,
       -5.74659966e+04, -5.34842643e+04, -4.96422919e+04, -4.59328291e+04,
       -4.23554062e+04, -3.89156402e+04, -3.56249648e+04, -3.24689676e+04,
       -2.94405370e+04, -2.65670654e+04, -2.38448281e+04, -2.12591793e+04,
       -1.88167910e+04, -1.65273839e+04, -1.44006735e+04, -1.24189630e+04,
       -1.05866827e+04, -8.91302175e+03, -7.38727441e+03, -5.99546084e+03,
       -4.74984593e+03, -3.66291401e+03, -2.72212548e+03, -8.40075923e+03,
       -4.54697956e+02, -5.03875390e+02, -5.48403162e+01,  1.37503695e+03,
        3.14605030e+02,  1.71395354e+01, -2.96597224e+02, -4.61721973e+02,
        7.11287974e+01, -5.79322735e+02, -9.41121962e+02, -4.45072965e+03,
       -9.78815772e+02,  4.30309901e+00, -3.50223313e+01,  6.18197577e+02,
       -1.04390270e+03,  1.17041986e+02,  9.52587351e+02])

In [4]:
cpp_grad # C++ gradient

array([-7.01413296e+04, -1.38276722e+05, -6.57712430e+04, -6.15535990e+04,
       -5.74459966e+04, -5.34642643e+04, -4.96222919e+04, -4.59128291e+04,
       -4.23354062e+04, -3.88956402e+04, -3.56049648e+04, -3.24489676e+04,
       -2.94205370e+04, -2.65470654e+04, -2.38248281e+04, -2.12391793e+04,
       -1.87967910e+04, -1.65073839e+04, -1.43806735e+04, -1.23989630e+04,
       -1.05666827e+04, -8.89302175e+03, -7.36727441e+03, -5.97546084e+03,
       -4.72984593e+03, -3.64291401e+03, -2.70212548e+03, -8.40075923e+03,
       -5.79322735e+02, -4.54697956e+02, -9.41121962e+02, -5.03875390e+02,
       -4.45072965e+03, -5.48403162e+01, -9.78815772e+02,  1.37503695e+03,
        4.30309901e+00,  3.14605030e+02, -3.50223313e+01,  1.71395354e+01,
        6.18197577e+02, -2.96597224e+02, -1.04390270e+03, -4.61721973e+02,
        1.17041986e+02,  7.11287974e+01,  9.52587351e+02])

In [5]:
are_close = np.isclose(numerical_grad, python_grad, rtol=1e-1)
are_close


array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [6]:
are_close = np.isclose(python_grad[2:27], cpp_grad[2:27], rtol=1e-1) # delta
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [7]:
are_close = np.isclose(numerical_grad[2:27], cpp_grad[2:27], rtol=1e-1) # delta
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True])

In [8]:
are_close = np.isclose(numerical_grad[-20:], python_grad[-20:], rtol=1e-1)
are_close

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [11]:
are_close = np.isclose(python_grad[-20:], cpp_grad[-20:], rtol=1e-1)
are_close

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True])

In [12]:
are_close = np.isclose(numerical_grad[-20:], cpp_grad[-20:], rtol=1e-1)
are_close

array([ True, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False,  True])

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_5.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
cpp_grad = np.ascontiguousarray(gradient, dtype=np.float64)

# Call the C++ function
lib.gradient(
    params_ctypes, len(params_ctypes),
    t_scaled_ctypes, len(t_scaled_ctypes),
    change_points_ctypes, len(change_points_ctypes),
    model.scale_period,
    normalized_y_ctypes, len(normalized_y_ctypes),
    model.sigma_obs,
    model.sigma_k,
    model.sigma_m,
    model.sigma,
    model.tau,
    cpp_grad
)

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_7.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
cpp_grad = np.ascontiguousarray(gradient, dtype=np.float64)
try:
    # Call the C++ function
    lib.gradient(
        params_ctypes, len(params_ctypes),
        t_scaled_ctypes, len(t_scaled_ctypes),
        change_points_ctypes, len(change_points_ctypes),
        model.scale_period,
        normalized_y_ctypes, len(normalized_y_ctypes),
        model.sigma_obs,
        model.sigma_k,
        model.sigma_m,
        model.sigma,
        model.tau,
        cpp_grad
    )
except Exception as e:
    print(f"Error: {e}")

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    print("r size (Python)", r.shape)
    print("x size (Python)", x.shape)
    print("dbeta size (Python)", dbeta.shape)
    print("x matrix (Python)\n", x)
    print("r vector (Python)\n", r)
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

numerical_grad = numerical_gradient(loss, model, params)
python_grad = gradient(model, params)

: 

In [1]:
import numpy as np

In [1]:
import numpy as np
import pandas as pd
import ctypes
from customProphet6 import *

# Load the shared library
lib = ctypes.CDLL('./libgradient_7.so')

# Define argument and return types
lib.gradient.argtypes = [
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS'),
    ctypes.c_size_t,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    ctypes.c_double,
    np.ctypeslib.ndpointer(dtype=np.float64, ndim=1, flags='C_CONTIGUOUS')
]

# Load data
df = pd.read_csv('peyton_manning.csv')

# Instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

# Convert data to ctypes
params_ctypes = np.ascontiguousarray(params, dtype=np.float64)
t_scaled_ctypes = np.ascontiguousarray(model.t_scaled, dtype=np.float64)
change_points_ctypes = np.ascontiguousarray(model.change_points, dtype=np.float64)
normalized_y_ctypes = np.ascontiguousarray(model.normalized_y, dtype=np.float64)

# Create an array to store the gradient
gradient = np.zeros(params.size, dtype=np.float64)
cpp_grad = np.ascontiguousarray(gradient, dtype=np.float64)
try:
    # Call the C++ function
    lib.gradient(
        params_ctypes, len(params_ctypes),
        t_scaled_ctypes, len(t_scaled_ctypes),
        change_points_ctypes, len(change_points_ctypes),
        model.scale_period,
        normalized_y_ctypes, len(normalized_y_ctypes),
        model.sigma_obs,
        model.sigma_k,
        model.sigma_m,
        model.sigma,
        model.tau,
        cpp_grad
    )
except Exception as e:
    print(f"Error: {e}")

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    #print("r size (Python)", r.shape)
    #print("x size (Python)", x.shape)
    #print("dbeta size (Python)", dbeta.shape)
    #print("r vector (Python)\n", r)
    #print("first row of x matrix (Python)\n", x[0])
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

numerical_grad = numerical_gradient(loss, model, params)
python_grad = gradient(model, params)

In [2]:
numerical_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944783e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377440e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035575e+03, -3.01457327e+03, -3.28097289e+02,  8.22653715e+03,
        1.88221122e+03,  1.02541992e+02, -1.77447454e+03, -2.76237883e+03,
        4.25547623e+02, -3.46595777e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603487e+03,  2.57444917e+01, -2.09530757e+02,  3.69853724e+03,
       -6.24543533e+03,  7.00235891e+02,  5.69911610e+03])

In [3]:
python_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944784e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377439e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,  8.22653712e+03,
        1.88221121e+03,  1.02541989e+02, -1.77447455e+03, -2.76237882e+03,
        4.25547613e+02, -3.46595776e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,  3.69853720e+03,
       -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [4]:
cpp_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944784e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377439e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,  8.22653712e+03,
        1.88221121e+03,  1.02541989e+02, -1.77447455e+03, -2.76237882e+03,
        4.25547613e+02, -3.46595776e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,  3.69853720e+03,
       -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [5]:
numerical_grad == python_grad

array([False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False, False, False, False, False, False, False, False,
       False, False])

In [6]:
np.all(numerical_grad - python_grad < 1e-4)

True

In [7]:
cpp_grad - python_grad

array([ 0.00000000e+00, -3.49245965e-10,  1.10594556e-09, -1.74622983e-10,
        2.91038305e-10,  1.74622983e-10, -5.82076609e-11,  4.07453626e-10,
        2.03726813e-10,  2.91038305e-10,  1.16415322e-10, -1.16415322e-10,
        1.16415322e-10, -8.73114914e-11,  2.91038305e-11,  1.60071068e-10,
        0.00000000e+00,  1.16415322e-10, -1.30967237e-10,  2.91038305e-11,
       -5.82076609e-11, -6.54836185e-11,  1.45519152e-11,  2.91038305e-11,
        1.09139364e-11, -1.45519152e-11, -9.09494702e-12,  5.82076609e-11,
       -3.00133252e-11,  2.44654075e-10, -1.35287337e-11, -1.27329258e-11,
       -2.00543582e-10, -1.04918740e-10, -6.82121026e-13, -1.09139364e-11,
       -7.56017471e-12, -5.04769559e-11,  1.27329258e-11, -4.36557457e-11,
        1.81898940e-12, -6.30961949e-12,  1.50521373e-10, -1.22327037e-10,
       -2.72848411e-12, -5.00222086e-12, -5.45696821e-12])

In [8]:
numerical_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944783e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377440e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035575e+03, -3.01457327e+03, -3.28097289e+02,  8.22653715e+03,
        1.88221122e+03,  1.02541992e+02, -1.77447454e+03, -2.76237883e+03,
        4.25547623e+02, -3.46595777e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603487e+03,  2.57444917e+01, -2.09530757e+02,  3.69853724e+03,
       -6.24543533e+03,  7.00235891e+02,  5.69911610e+03])

In [9]:
python_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944784e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377439e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,  8.22653712e+03,
        1.88221121e+03,  1.02541989e+02, -1.77447455e+03, -2.76237882e+03,
        4.25547613e+02, -3.46595776e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,  3.69853720e+03,
       -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [10]:
cpp_grad

array([-4.19639816e+05, -8.27278558e+05, -3.93614225e+05, -3.68381009e+05,
       -3.43806146e+05, -3.19984337e+05, -2.96998679e+05, -2.74805796e+05,
       -2.53402879e+05, -2.32823532e+05, -2.13136160e+05, -1.94254538e+05,
       -1.76136118e+05, -1.58944784e+05, -1.42658249e+05, -1.27188893e+05,
       -1.12576633e+05, -9.88796243e+04, -8.61559943e+04, -7.42998653e+04,
       -6.33377439e+04, -5.33246065e+04, -4.41964031e+04, -3.58694952e+04,
       -2.84172610e+04, -2.19143915e+04, -1.62858651e+04, -5.02598550e+04,
       -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,  8.22653712e+03,
        1.88221121e+03,  1.02541989e+02, -1.77447455e+03, -2.76237882e+03,
        4.25547613e+02, -3.46595776e+03, -5.63052125e+03, -2.66277155e+04,
       -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,  3.69853720e+03,
       -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [11]:
cpp_grad[-20:]

array([-5.02598550e+04, -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,
        8.22653712e+03,  1.88221121e+03,  1.02541989e+02, -1.77447455e+03,
       -2.76237882e+03,  4.25547613e+02, -3.46595776e+03, -5.63052125e+03,
       -2.66277155e+04, -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,
        3.69853720e+03, -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [12]:
python_grad[-20:]

array([-5.02598550e+04, -2.72035571e+03, -3.01457325e+03, -3.28097291e+02,
        8.22653712e+03,  1.88221121e+03,  1.02541989e+02, -1.77447455e+03,
       -2.76237882e+03,  4.25547613e+02, -3.46595776e+03, -5.63052125e+03,
       -2.66277155e+04, -5.85603484e+03,  2.57444746e+01, -2.09530739e+02,
        3.69853720e+03, -6.24543533e+03,  7.00235908e+02,  5.69911609e+03])

In [13]:
abs(cpp_grad - python_grad) < 1e-2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [14]:
abs(cpp_grad - python_grad)[-20:] < 1e-2

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [15]:
np.all((abs(cpp_grad - python_grad))[2:27] < 1e-3) # is delta close?

True

In [16]:
np.all((abs(cpp_grad - python_grad))[-20:] < 1e-3) # is beta close?

True

In [17]:
abs((cpp_grad - python_grad))[-20:]

array([5.82076609e-11, 3.00133252e-11, 2.44654075e-10, 1.35287337e-11,
       1.27329258e-11, 2.00543582e-10, 1.04918740e-10, 6.82121026e-13,
       1.09139364e-11, 7.56017471e-12, 5.04769559e-11, 1.27329258e-11,
       4.36557457e-11, 1.81898940e-12, 6.30961949e-12, 1.50521373e-10,
       1.22327037e-10, 2.72848411e-12, 5.00222086e-12, 5.45696821e-12])

In [18]:
abs((cpp_grad - numerical_grad))[-20:]

array([7.68762402e-06, 4.61828922e-05, 2.55524510e-05, 1.87759878e-06,
       2.22810049e-05, 1.03358448e-05, 2.10066275e-06, 7.75715853e-06,
       9.68376753e-06, 9.82972460e-06, 3.30662260e-07, 1.97808367e-06,
       1.79270901e-05, 2.63947595e-05, 1.71556371e-05, 1.80252629e-05,
       3.76264279e-05, 2.60656088e-06, 1.69758596e-05, 2.76243554e-06])

In [19]:
abs((python_grad - numerical_grad))[-20:]

array([1.00645025e-07, 3.27017041e-07, 3.02018648e-07, 2.62901318e-08,
       4.46495960e-07, 2.85752115e-07, 1.68573262e-08, 2.50210729e-07,
       1.92382771e-07, 1.85797883e-07, 1.93269756e-08, 1.05879906e-07,
       9.79495667e-08, 3.15152420e-08, 1.96342142e-07, 9.31424275e-08,
       2.89983639e-07, 4.38550543e-07, 2.31753246e-07, 1.55565928e-07])

In [14]:
from autograd import grad
import autograd.numpy as np  # Use autograd's wrapped numpy

def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        + np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        + m**2 / (2 * model.sigma_m**2)
        
def gradient(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + np.dot(A, delta)) * model.t_scaled + (m + np.dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    dk = np.array([-np.sum(r * model.t_scaled) / model.sigma_obs**2 + k / model.sigma_k**2])
    dm = np.array([-np.sum(r) / model.sigma_obs**2 + m / model.sigma_m**2])
    ddelta = -np.sum(r[:, None] * (model.t_scaled[:, None] - model.change_points) * A, axis=0) / model.sigma_obs**2 + np.sign(delta) / model.tau
    dbeta = -np.dot(r, x) / model.sigma_obs**2 + beta / model.sigma**2
    
    return np.concatenate([dk, dm, ddelta, dbeta])

def numerical_gradient(loss, model, params, epsilon=1e-6):
    num_grad = np.zeros_like(params)
    for i in range(len(params)):
        params_perturbed = params.copy()
        params_perturbed[i] += epsilon
        loss_plus_epsilon = loss(model, params_perturbed)
        
        params_perturbed[i] -= 2 * epsilon
        loss_minus_epsilon = loss(model, params_perturbed)
        
        num_grad[i] = (loss_plus_epsilon - loss_minus_epsilon) / (2 * epsilon)
    
    return num_grad

## instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))

numerical_grad = numerical_gradient(loss, model, params)
python_grad = model._gradient(params)

grad_loss = grad(model._gradient)
auto_grad = grad_loss(params)


TypeError: grad_np_sum() got an unexpected keyword argument 'out'

In [20]:
auto_grad - cpp_grad

array([ 2.86762412e+06,  5.65323846e+06,  2.68977729e+06,  2.51734517e+06,
        2.34941195e+06,  2.18662474e+06,  2.02955140e+06,  1.87789552e+06,
        1.73163790e+06,  1.59100817e+06,  1.45647379e+06,  1.32744553e+06,
        1.20363264e+06,  1.08615503e+06,  9.74860393e+05,  8.69149981e+05,
        7.69296562e+05,  6.75697550e+05,  5.88750157e+05,  5.07730862e+05,
        4.32820803e+05,  3.64395660e+05,  3.02017746e+05,  2.45115515e+05,
        1.94190399e+05,  1.49752801e+05,  1.11290058e+05,  3.43452568e+05,
        2.45302197e+04,  1.82560251e+04,  4.44890176e+04,  3.33469693e+04,
        2.14290089e+05,  2.73038506e+03,  4.46456840e+04, -6.86771422e+04,
        2.77427933e+02, -1.89266753e+04, -4.71525519e+03, -3.10112979e+04,
       -3.61084248e+04,  1.41672819e+04,  4.95227805e+04,  2.62031242e+04,
       -1.26610792e+04, -2.59651456e+03, -3.89451195e+04])

In [14]:
vec = np.array([1, 2, 3])
mat = np.array([[1, 4, 7],
                [2, 5, 8],
                [3, 6, 9]])

result = np.dot(vec, mat)
print(result)

[14 32 50]


In [18]:
def loss(model, params):
    k, m, delta, beta = extract_params(params)
    
    A = (model.t_scaled[:, None] > model.change_points) * 1
    gamma = -model.change_points * delta
    g = (k + det_dot(A, delta)) * model.t_scaled + (m + det_dot(A, gamma))

    period = 365.25 / model.scale_period
    x = fourier_components(model.t_scaled, period, 10)
    s = np.dot(x, beta)

    y_pred = g + s
    y_true = model.normalized_y
    r = y_true - y_pred
    
    return x, r
    #return np.sum(r**2) / (2 * model.sigma_obs**2) + np.sum(beta**2) / 2*model.sigma**2 \
        #+ np.sum(np.abs(delta)) / model.tau + k**2 / (2 * model.sigma_k**2) \
        #+ m**2 / (2 * model.sigma_m**2)
        
        
## instantiate & initialize a model
model = CustomProphet()
model.y = df['y'].values
if df['ds'].dtype != 'datetime64[ns]':
    model.ds = pd.to_datetime(df['ds'])
else:
    model.ds = df['ds']

model.t_scaled = np.array((model.ds - model.ds.min()) / (model.ds.max() - model.ds.min()))
model.T = df.shape[0]

model.scale_period = (model.ds.max() - model.ds.min()).days
model._normalize_y()
model._generate_change_points()

params = np.zeros((47,))
x, r = loss(model, params)
x.shape, r.shape


((2905, 20), (2905,))

In [20]:
np.dot(x.T, r) == np.dot(r, x)

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])

In [21]:
np.dot(r.T, x) == np.dot(r, x)
x = 

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True])